
---

## 🧠 **LangChain & LangGraph Integration**

| Topic                            | Purpose / Usage                                                                                                                        |
| -------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------- |
| **Installation & Setup**         | `pip install streamlit langchain langgraph openai` (plus any connectors, e.g. `langchain-openai`) ([Streamlit Docs][1])                |
| **Streamlit Chat UI**            | `st.chat_input()` + `st.chat_message()` to build interactive chatbot interfaces ([Streamlit Docs][2])                                  |
| **Chains & Agents**              | Design LangChain chains or LangGraph agents triggered via Streamlit UI elements ([GitHub][3], [KDnuggets][4])                          |
| **Conversation Memory**          | Use `StreamlitChatMessageHistory` or LangChain memory classes with session state to persist chat history ([LangChain][5], [Reddit][6]) |
| **Callback Visualization**       | Use `StreamlitCallbackHandler` to render chain actions/LLM thoughts step-by-step                                                       |
| **Chain Graph Rendering**        | Visualize LangGraph workflows using `st.graphviz_chart()` or pre-generated diagrams via `st.image()`                                   |
| **Tool Use & RAG**               | Support file/document QA via upload → embedding → retrieval pipelines (e.g. Deepseek, Deepseek with audio, vector databases)           |
| **Routing Logic with LangGraph** | Use LangGraph nodes to build conditional routing, multi-tool orchestration, triggered from UI events                                   |
| **Debugging & Tracing Tools**    | Enable inspection using `st.expander()`, `st.json()`, or logs to examine agent decisions and traces                                    |
| **Async / FastAPI Integration**  | For LangGraph workflows, route through FastAPI to manage streaming or async execution; UI then consumes via Streamlit app              |



---



In [ ]:
#📄 2. app.py – Streamlit + LangChain Chatbot
import streamlit as st
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.callbacks.streamlit import StreamlitCallbackHandler
from langchain_core.prompts import PromptTemplate
import os

# --- PAGE CONFIG ---
st.set_page_config(page_title="🧠 LangChain Chatbot", layout="centered")
st.title("💬 LangChain + Streamlit Chat")

# --- API KEY ---
openai_key = st.secrets["OPENAI_API_KEY"]
os.environ["OPENAI_API_KEY"] = openai_key

# --- SESSION STATE INIT ---
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# --- INPUT ---
user_input = st.chat_input("Say something...")

# --- DISPLAY PAST MESSAGES ---
for role, msg in st.session_state.chat_history:
    with st.chat_message(role):
        st.markdown(msg)

# --- PROCESS INPUT ---
if user_input:
    # Show user message
    st.session_state.chat_history.append(("user", user_input))
    with st.chat_message("user"):
        st.markdown(user_input)

    # LangChain Memory
    memory = ConversationBufferMemory(return_messages=True)

    # Chat Model
    llm = ChatOpenAI(temperature=0)

    # Prompt (optional – shows how to inject your own)
    prompt = PromptTemplate(
        input_variables=["history", "input"],
        template="You are a helpful assistant.\nChat History:\n{history}\nUser: {input}\nAssistant:"
    )

    # Callback handler (see step-by-step trace)
    callback = StreamlitCallbackHandler(parent_container=st.container())

    # LangChain Chain
    chain = ConversationChain(
        llm=llm,
        memory=memory,
        prompt=prompt,
        verbose=True,
        callbacks=[callback],
    )

    # Get response
    response = chain.run(input=user_input)

    # Show assistant reply
    st.session_state.chat_history.append(("assistant", response))
    with st.chat_message("assistant"):
        st.markdown(response)


In [ ]:
# 📄 2. Define Your LangGraph Workflow – graph_workflow.py

from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI

# Simple graph node: LLM replies with a basic answer
def reply_node(state):
    messages = state.get("messages", [])
    chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    response = chat.invoke(messages)
    messages.append(response)
    return {"messages": messages}

# Define the LangGraph workflow
def build_conversation_graph():
    builder = StateGraph(schema={"messages": list})
    
    builder.add_node("respond", RunnableLambda(reply_node))
    builder.set_entry_point("respond")
    builder.set_finish_point("respond")  # No loops for now

    return builder.compile()



# 3. Build Streamlit UI – app.py

import streamlit as st
from langgraph.graph import MessageGraph
from langchain_core.messages import HumanMessage
from graph_workflow import build_conversation_graph

# Page setup
st.set_page_config(page_title="LangGraph Chat", layout="centered")
st.title("🤖 LangGraph + Streamlit")

# Initialize state
if "history" not in st.session_state:
    st.session_state.history = []
    st.session_state.graph = build_conversation_graph()
    st.session_state.runner = st.session_state.graph.invoke

# User input
user_input = st.chat_input("Type your message")

# Display past messages
for msg in st.session_state.history:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Handle new message
if user_input:
    # Show user message
    st.session_state.history.append({"role": "user", "content": user_input})
    with st.chat_message("user"):
        st.markdown(user_input)

    # Create message list for LangGraph
    messages = [HumanMessage(content=user_input)]

    # Run the LangGraph workflow
    result = st.session_state.runner({"messages": messages})
    response_msg = result["messages"][-1]

    # Show assistant response
    st.session_state.history.append({"role": "assistant", "content": response_msg.content})
    with st.chat_message("assistant"):
        st.markdown(response_msg.content)
